A simple Twitter comments and Google finance Data scrapping to predict the future stock price of a quote

In [1]:
#importing all the required Libraries
import tweepy, requests
import numpy as np

from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from textblob import TextBlob
%matplotlib inline

In [2]:
# Organising for the keys, File save path
consumer_key = 'cBacprmur1W92tY4uoeZWviGA'
consumer_secret = '0nRQQNC5aeX7XBEqIVVJDni6w4HIPVNj6cA0JwVy4faAsQBKO0'
access_token = '371711493-efb1jvsAlV13mHxX8J7y36FubdeP6QH3eyqttVxw'
access_token_secret = 'vrbDkAVTcRmk1c2EoPyI6S90cjg6sIQrmkXaY0X7fRje7'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
user = tweepy.API(auth)

# file to download the Data of stock prices If available
FILE_NAME = 'Stock_Data.csv'

In [3]:
# Verify for the common sentiment of the stock and return Good/Bad accordingly
# Based on subjectivity and polarity of a single tweet and further their average

def get_sentiment(name, num_tweets):
    list_of_tweets = user.search(name, count=num_tweets)
    positive, null = 0, 0

    for tweet in list_of_tweets:
        blob = TextBlob(tweet.text).sentiment
        if blob.subjectivity == 0:
            null += 1
            next
        if blob.polarity > 0:
            positive += 1
    #print(positive, null)
    if positive > ((num_tweets - null) / 2):
        return 'Good'
    return 'Bad'

In [ ]:
Using Textblob and after retrieving the somment / statement froim twitter we calculate teh polarity and subjectivity of the comment.
Further concluding it as Good/Bad comment - A very nai

In [4]:
# Download the stock price data from Google Finance / Yahooo Finance and saves it to a file locally
def get_stockdata(name):
    url = 'http://www.google.com/finance/historical?q=NASDAQ%3A' + name + '&output=csv'
    r = requests.get(url, stream=True)

    if r.status_code != 400:
        with open(FILE_NAME, 'wb') as f:
            for chunk in r:
                f.write(chunk)

        return True

# Loading data from CSV and making a dataset of today's and Tomarrrows Stock prices for model fitting/estimation

def stock_prediction():
    dataset = []

    with open(FILE_NAME) as f:
        for n, line in enumerate(f):
            if n != 0:
                dataset.append(float(line.split(',')[1]))

    dataset = np.array(dataset)

    # Create dataset matrix (X=today's price and Y=tomorow's price)
    def create_dataset(dataset):
        X = [dataset[n + 1] for n in range(len(dataset) - 2)]
        return np.array(X), dataset[2:]

    X, Y = create_dataset(dataset)
    X = X.reshape(-1, 1)
    Y = Y.reshape(-1, 1)

    # different models for Evaluation of the Regression function
    models = {
        'LinearReg': linear_model.LinearRegression(),
        'RidgeReg': linear_model.Ridge(alpha=.5),
        #'RidgeCV': linear_model.RidgeCV(alphas=[0.1, 1.0, 10.0]),
        'LasoReg': linear_model.Lasso(alpha=0.1),
        'LarsReg': linear_model.LassoLars(alpha=.1),
        'bayeReg': linear_model.BayesianRidge()
        # 'regsvr'   : SVR(C=1.0, epsilon=0.2)
    }

    # Cross Validation in 5 folds for the Proper Metric evaluation of the selected Model
    # Better the score --> Better the model works on the Test/New Data

    def cross_score(func, a, b):
        scores = cross_val_score(func, a, b, cv=5)
        return scores.mean(), scores.std() * 2

    # Iterate through all the available models and output the predictions with Accuracy to the console
    for model, func in models.items():
        func.fit(X, Y.ravel())
        scr = cross_score(func, X, Y.ravel())
        pred = func.predict(dataset[0]).item()
        print('{:10}--> CV-accuracy: {:.2f} (+/- {:.2f}) prediction: {} ~ {}'.format(model, scr[0], scr[1],
                                                                                     round(dataset[0], 2),
                                                                                     round(pred, 2)))


Above code uses different Regression - Supervised Learning algorithms and calculates their accuraries accordingly
The accuracy is calculated over training the Sets in cross validated phenomenon, 
i,e the train - test split is continuously evaluated with different folds (5 here) to obtrain max. efficiency


In [5]:
global name
# Entering the Stock Name
name = input('Enter a stock from NASDAQ (e.j: AAPL, FB, GOOGL): ').upper()


# Check if the stock sentiment is positve
def sentiment():
    stock_senti = get_sentiment(name, num_tweets=200)
    print('This stock has relatively {} sentiment on Twitter\n'.format(stock_senti))

sentiment()
# Check if we got teh historical stock price data
while not get_stockdata(name):
    print('Also, Google returned an error: Pls enter a valid stock Type')
    name = input('Enter a stock from NASDAQ: \n').upper()
    sentiment()
# We have our file so we create the neural net and get the prediction
stock_prediction()

Enter a stock from NASDAQ (e.j: AAPL, FB, GOOGL): spiderman
This stock has relatively Bad sentiment on Twitter

Also, Google returned an error: Pls enter a valid stock Type
Enter a stock from NASDAQ: 
AAPL
This stock has relatively Bad sentiment on Twitter

LinearReg --> CV-accuracy: 0.89 (+/- 0.10) prediction: 163.8 ~ 163.44
RidgeReg  --> CV-accuracy: 0.89 (+/- 0.10) prediction: 163.8 ~ 163.44
LasoReg   --> CV-accuracy: 0.89 (+/- 0.10) prediction: 163.8 ~ 163.43
LarsReg   --> CV-accuracy: 0.72 (+/- 0.31) prediction: 163.8 ~ 160.64
bayeReg   --> CV-accuracy: 0.89 (+/- 0.10) prediction: 163.8 ~ 163.44
